# Homework #3 by Tasneem Abbasi, Myles Cook, and Keegan Smith

In [1]:
import pandas as pd
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
auctions_df = pd.read_csv('eBayAuctions.csv')
auctions_df.head()

,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive?
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0


In [3]:
auctions_df.dropna(inplace=True)

auctions_df['Category'] = auctions_df['Category'].astype('category')
auctions_df['currency'] = auctions_df['currency'].astype('category')
auctions_df['endDay'] = auctions_df['endDay'].astype('category')
auctions_df['Duration'] = auctions_df['Duration'].astype('category')

cat_predictors = ['Category', 'currency', 'Duration', 'endDay']
numeric_predictors = ['sellerRating', 'ClosePrice', 'OpenPrice']
outcome = 'Competitive?'

X_cat = pd.get_dummies(auctions_df[cat_predictors], drop_first=True, dtype=int)

X = pd.concat([X_cat, auctions_df[numeric_predictors]], axis=1)
y = auctions_df['Competitive?']

# Problem #1a:

In [4]:
# Define the categorical variables
categorical_vars = ['Category', 'currency', 'endDay', 'Duration']

# Loop over each categorical variable
for var in categorical_vars:
    # Create a pivot table for each variable, calculating the sum of 'Competitive?' for each category
    pivot = auctions_df.pivot_table(index=var, values='Competitive?', aggfunc='sum')
    
    # Normalize the pivot table by dividing each value by the total sum
    pivot = pivot / pivot.sum()
    
    # Sort the pivot table in descending order based on 'Competitive?'
    pivot = pivot.sort_values(by='Competitive?', ascending=False)
    
    # Print the pivot table for each variable
    print(f'{var}:\n{pivot}\n')

Category:
                      Competitive?
Category                          
Music/Movie/Game          0.227955
Collectibles              0.129456
Toys/Hobbies              0.116323
Antique/Art/Craft         0.093809
SportingGoods             0.084428
Home/Garden               0.062852
Automotive                0.059099
Clothing/Accessories      0.056285
Electronics               0.041276
Jewelry                   0.028143
Books                     0.025328
Computer                  0.022514
Business/Industrial       0.011257
Coins/Stamps              0.010319
Health/Beauty             0.010319
Photography               0.010319
Pottery/Glass             0.006567
EverythingElse            0.003752

currency:
          Competitive?
currency              
US            0.629456
EUR           0.275797
GBP           0.094747

endDay:
        Competitive?
endDay              
Mon         0.346154
Sun         0.153846
Sat         0.140713
Fri         0.125704
Thu         0.114447
Tue     

# Problem #1b:

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

scaler = MinMaxScaler()
X_train_nd = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_nd, columns=X.columns, index=X_train.index)

scaler.fit(X_valid)
X_valid_nd = scaler.transform(X_valid)
X_valid = pd.DataFrame(X_valid_nd, columns=X.columns, index=X_valid.index)

model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)

# Predict on validation set
y_pred = model.predict(X_valid)

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy of logistic model with all predictors/features:", accuracy)

Accuracy of logistic model with all predictors/features: 0.6692015209125475


In [6]:
print(type(X_train))

<class 'pandas.core.frame.DataFrame'>


In [7]:
print(X_train)

      Category_Automotive  Category_Books  Category_Business/Industrial  \
503                   0.0             0.0                           0.0   
733                   0.0             0.0                           0.0   
383                   0.0             0.0                           0.0   
725                   0.0             0.0                           0.0   
310                   0.0             0.0                           0.0   
...                   ...             ...                           ...   
1791                  1.0             0.0                           0.0   
1096                  0.0             0.0                           0.0   
1932                  0.0             0.0                           0.0   
235                   0.0             0.0                           0.0   
1061                  0.0             0.0                           0.0   

      Category_Clothing/Accessories  Category_Coins/Stamps  \
503                             0.0  

# Problem #1c:

In [8]:
X_alt = X.drop(columns='ClosePrice')

In [9]:
X_train_alt, X_valid_alt, y_train_alt, y_valid_alt = train_test_split(X_alt, y, test_size=0.4, random_state=42)

X_train_alt = scaler.fit_transform(X_train_alt)
X_valid_alt = scaler.transform(X_valid_alt)

model_no_price = LogisticRegression()
model_no_price.fit(X_train_alt, y_train_alt)

y_pred_no_price = model_no_price.predict(X_valid_alt)

accuracy_no_price = accuracy_score(y_valid_alt, y_pred_no_price)
print("Accuracy of logistic model excluding price:", accuracy_no_price)

Accuracy of logistic model excluding price: 0.6476552598225602


Less accurate 

# Problem #1d:

In [10]:
import statsmodels.api as sm
model_eval = sm.Logit(y_train, sm.add_constant(X_train))
model_eval = model_eval.fit()
print(model_eval.summary())

         Current function value: 0.499076
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:           Competitive?   No. Observations:                 1183
Model:                          Logit   Df Residuals:                     1150
Method:                           MLE   Df Model:                           32
Date:                Tue, 27 Feb 2024   Pseudo R-squ.:                  0.2778
Time:                        23:21:40   Log-Likelihood:                -590.41
converged:                      False   LL-Null:                       -817.49
Covariance Type:            nonrobust   LLR p-value:                 4.332e-76
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                            -0.7236      0.941     -0.769      0.442      -2.569       1.121
Category

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


The log probability of a competetive auction is increased by 0.0894 for each 1 unit (dollar) of closing price and it is statistically significant (p < 0.1)

# Problem #1e: 

In [11]:
model_regularized = LogisticRegression(penalty='l1', solver='liblinear')
model_regularized.fit(X_train, y_train)

# Predict on validation set
y_pred_regularized = model_regularized.predict(X_valid)

# Calculate accuracy
accuracy_regularized = accuracy_score(y_valid, y_pred_regularized)
print("Accuracy of regularized logistic model:", accuracy_regularized)

Accuracy of regularized logistic model: 0.6894803548795945


# Problem #1f:

We would recommend an auction with a duration of 5 days ending on either Friday or Monday given that those have the highest possitive coefficients, and that the auction use British Pounds (GBP) and is in the category "photography" although the reliance of these data is undetermined in our model. 